# 📦 Supply Chain Inventory Optimization & Supplier Performance Dashboard
## 🧩 Advanced EDA + Statistical Tests with Business Insights
This notebook demonstrates real EDA with actionable insights for supply chain and supplier performance improvements.

In [ ]:
# 📚 Import Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
sns.set(style='whitegrid')

In [ ]:
# 📂 Load Data
products = pd.read_csv('products.csv')
suppliers = pd.read_csv('suppliers.csv')
warehouses = pd.read_csv('warehouses.csv')
inventory = pd.read_csv('inventory_snapshots.csv')
orders = pd.read_csv('orders.csv')

In [ ]:
# 🔗 Merge Data
merged = inventory.merge(products, on='ProductID').merge(suppliers, on='SupplierID')
merged['SnapshotDate'] = pd.to_datetime(merged['SnapshotDate'])
merged['StockRisk'] = np.where(merged['StockLevel'] < merged['ReorderPoint'], 1, 0)
merged.head()

### ✅ **Observation:**
Data merged successfully. Added `StockRisk` flag for items below reorder point.

In [ ]:
# 📊 Total Stock Level Trend
stock_trend = merged.groupby('SnapshotDate')['StockLevel'].sum().reset_index()
plt.figure(figsize=(12,6))
sns.lineplot(data=stock_trend, x='SnapshotDate', y='StockLevel')
plt.title('📈 Total Stock Level Trend Over Time')
plt.show()

**Insight:** Total stock levels show seasonality and fluctuations. A downward trend could imply supply risk — check reorder strategies.

In [ ]:
# 🔍 Monthly Stockout Rate
stockout_rate = merged.groupby(['SnapshotDate'])['StockRisk'].mean().reset_index()
plt.figure(figsize=(12,6))
sns.lineplot(data=stockout_rate, x='SnapshotDate', y='StockRisk')
plt.title('📉 Monthly Stockout Rate Trend')
plt.show()

**Insight:** Higher stockout rates in certain months suggest supply chain bottlenecks. This insight helps plan buffer stock.

In [ ]:
# 🔍 Top 5 Suppliers by Late Deliveries
orders['LateFlag'] = np.where(
    pd.to_datetime(orders['ActualDeliveryDate']) > pd.to_datetime(orders['PlannedDeliveryDate']), 1, 0)
late_by_supplier = orders.groupby('SupplierID')['LateFlag'].sum().reset_index()
late_by_supplier = late_by_supplier.merge(suppliers, on='SupplierID').sort_values('LateFlag', ascending=False).head(5)

plt.figure(figsize=(10,6))
sns.barplot(data=late_by_supplier, x='SupplierName', y='LateFlag')
plt.title('🚚 Top 5 Suppliers by Late Deliveries')
plt.show()

**Insight:** Focus supplier relationship management on these suppliers to reduce delays and improve fulfillment KPIs.

In [ ]:
# 📊 Category-wise Avg Lead Time
cat_leadtime = merged.groupby('Category')['AvgLeadTimeDays'].mean().reset_index()
plt.figure(figsize=(10,6))
sns.barplot(data=cat_leadtime, x='Category', y='AvgLeadTimeDays')
plt.title('⏱️ Average Lead Time by Product Category')
plt.show()

**Observation:** Electronics & Home items may have longer lead times — plan orders in advance to avoid stockouts.

In [ ]:
# 🔥 Chi-Square Test: Category vs Stock Risk
ct = pd.crosstab(merged['Category'], merged['StockRisk'])
chi2, p, dof, exp = stats.chi2_contingency(ct)
print(f'Chi-square p-value: {p}')

**Conclusion:** If p-value < 0.05, Category significantly impacts stockout risk. Revisit reorder points for high-risk categories.

In [ ]:
# ⚙️ ANOVA: Lead Time by Supplier
groups = [merged[merged['SupplierName']==name]['AvgLeadTimeDays'] for name in merged['SupplierName'].unique()]
anova = stats.f_oneway(*groups)
print(f'ANOVA p-value: {anova.pvalue}')

**Conclusion:** If p-value < 0.05, average lead times differ significantly across suppliers — negotiate SLAs with worst performers.

In [ ]:
# ✅ Save Clean Data
merged.to_csv('merged_clean.csv', index=False)
print('✔️ Cleaned dataset saved for Power BI / Excel.')

### 📌 Final Takeaways
- Stock levels fluctuate seasonally → build buffer stock plans.
- Electronics have higher stockout risk → adjust reorder policies.
- Some suppliers consistently deliver late → review contracts.
- Insights ready for Power BI Dashboard → for daily operational tracking.